## Overall Model Comparison

| Model        | Augmentation | BatchNorm | Conv Filters     | Dense Units | Params     | Train Acc | Val Acc | Overfitting | Notes |
|--------------|--------------|-----------|------------------|-------------|------------|-----------|---------|--------------|-------|
| **Base**     | No           | No        | 32 → 64 → 64     | 64          | ~1.4M      | 97.12%    | 84.72%  | High         | Overfitting visible |
| **Mod-1**    | No           | No        | 8 → 16 → 32      | 32          | ~330k      | 94.01%    | 81.62%  | Reduced      | Simpler, still overfitting |
| **Mod-2**    | No           | No        | 4 → 8 → 16       | 16          | ~84k       | 75.66%    | 75.95%  | None         | Very balanced, efficient |
| **Mod-3**    | Yes          | No        | 4 → 8 → 16       | 16          | ~84k       | 71.78%    | 75.19%  | None         | Augmentation helps generalization |
| **Mod-4**    | Yes          | Yes       | 8 → 16 → 32      | 32          | ~332k      | 78.97%    | 79.90%  | None         | Best generalization and lowest loss |

### Interpretation:
- **Best accuracy**: Base model — but suffers from clear overfitting
- **Best generalization**: Modified-4 — excellent validation performance without overfitting
- **Best efficiency**: Modified-2 — strong balance with only ~84k parameters
- **Most effective regularization**: BatchNorm + Augmentation (Modified-4)